# User params

In [1]:
n_data = 2  # numebr of samples

In [2]:
import os, sys
sys.path.append(os.path.abspath('../src'))
from src.config import BASE_PATH_DATA, PATH_PROJECT
os.chdir(PATH_PROJECT)

# Utility functions

# Prepare some data

In [3]:
from src.utils.io.io import load_dataframe
from src.config import BASE_PATH_DATA
from src.api.utils import create_batch_detection_request

# load test data
df_test = load_dataframe(os.path.join(BASE_PATH_DATA, 'processed', 'test.parquet'))
df_test = df_test.sample(n_data)  # take some samples
display(df_test)

# Print column names and their data types
for col in df_test.columns:
    print(f"{col}: {df_test[col].dtype}")

# format the data for a request
json_data = df_test.to_dict(orient="records")
payload = create_batch_detection_request(json_data)

print(payload.requests[0])
print(payload)


2024-08-08 08:34:17,177 - src.config - DEBUG - calling load_dataframe
2024-08-08 08:34:17,178 - src.config - INFO - Current working directory: C:\Users\alexi\WORKSPACE\interview_abwaab\ReviewOutliers


,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,description,price,images_metadata,videos,store,categories,details,bought_together,subtitle,author
11997,5.0,Perfect!,I accidentally burned a plastic spoon onto my ...,[],B003BNZ4Z4,B003BNZ954,AHNASNRZL6PBPRR6PQYLCA5PYJGQ,1654229300148,1,True,...,"[Product Description, This high quality Genuin...",37.06,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['Generic Version was shipped (not a...,GE,"[Appliances, Parts & Accessories, Range Parts ...","{""Manufacturer"": ""GE"", ""Part Number"": ""WB30M2""...",None,None,None
41594,5.0,Very happy.,As described. Very happy.,[],B00DZUAEVW,B00DZUAEVW,AHZOBCY2BXC2SWQV47QIDTJ57WRA,1463280089000,0,True,...,"[Product Description, This is a genuine replac...",13.69,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['My Icemaker Stopped Making Ice | K...,Whirlpool,"[Appliances, Parts & Accessories, Refrigerator...","{""Manufacturer"": ""Whirlpool"", ""Part Number"": ""...",None,None,None


2024-08-08 08:34:22,820 - src.config - DEBUG - calling create_batch_detection_request
2024-08-08 08:34:22,822 - src.config - DEBUG - calling create_detection_request
2024-08-08 08:34:22,823 - src.config - DEBUG - user_id found in input data
2024-08-08 08:34:22,824 - src.config - DEBUG - title_review found in input data
2024-08-08 08:34:22,826 - src.config - DEBUG - text found in input data
2024-08-08 08:34:22,828 - src.config - DEBUG - title_metadata found in input data
2024-08-08 08:34:22,829 - src.config - DEBUG - features found in input data
2024-08-08 08:34:22,831 - src.config - DEBUG - description found in input data
2024-08-08 08:34:22,832 - src.config - DEBUG - store found in input data
2024-08-08 08:34:22,835 - src.config - DEBUG - asin found in input data
2024-08-08 08:34:22,837 - src.config - DEBUG - parent_asin found in input data
2024-08-08 08:34:22,841 - src.config - DEBUG - timestamp found in input data
2024-08-08 08:34:22,843 - src.config - DEBUG - rating found in input 

rating: float64
title_review: object
text: object
images_review: object
asin: object
parent_asin: object
user_id: object
timestamp: int64
helpful_vote: int64
verified_purchase: bool
main_category: object
title_metadata: object
average_rating: float64
rating_number: int64
features: object
description: object
price: object
images_metadata: object
videos: object
store: object
categories: object
details: object
bought_together: object
subtitle: object
author: object
user_id='AHNASNRZL6PBPRR6PQYLCA5PYJGQ' title_review='Perfect!' text='I accidentally burned a plastic spoon onto my burner and this was a perfect replacement.  Came quick and was a perfect fit!' title_metadata='GE WB30M2 Genuine OEM 8" Surface Element Coil for GE Electric Ranges' features=['noture/20', 'Imported', 'The GE Appliances WB30M2 8" Surface Element Coil is a genuine OEM (Original Equipment Manufacturer) part designed and engineered to exact specifications', 'Replacement GE Appliances Range 8" Surface Element Coil suppl

# Launch the service

**In local**

In [12]:
%%sh
# check process is at project root (
# -> ok because changed dir at the beginning of the notebook
pwd

# Kill any existing Uvicorn processes
kill $(ps aux | grep 'uvicorn' | grep -v grep | awk '{print $2}')

# Start a new Uvicorn server process with nohup to detach it and run it in the background
# poetry run uvicorn src.app:app --host 0.0.0.0 --port 8000 --reload: Run the Uvicorn server using Poetry
# --host 0.0.0.0: Bind the server to all network interfaces
# --port 8000: Use port 8000 for the server
# --reload: Enable auto-reloading for code changes (useful during development)
# > logs/uvicorn.log 2>&1: Redirect both stdout and stderr to the log file logs/uvicorn.log
# &: Run the command in the background
nohup poetry run uvicorn src.app:app --host 0.0.0.0 --port 8000 --reload > logs/uvicorn.log 2>&1 &

# Wait for 5 seconds to ensure the server has started
sleep 5

# Check if the server is running by sending a request
# If curl fails (i.e., the server is not running or the endpoint is incorrect), it outputs "Server is not running".
curl -f http://localhost:8000/ || echo "Server is not running"



/c/Users/alexi/WORKSPACE/interview_abwaab/ReviewOutliers


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0    369      0 --:--:-- --:--:-- --:--:--   369


{"message":"Welcome 👋 to the Outlier Detection and Distribution Shift API"}

**Using docker**

In [ ]:
%%sh

# Kill any existing Uvicorn processes
kill $(ps aux | grep 'uvicorn' | grep -v grep | awk '{print $2}')

# build the container
make build_docker

# run it
make run_docker

# Wait for 5 seconds to ensure the server has started
sleep 5

# Check if the server is running by sending a request
# If curl fails (i.e., the server is not running or the endpoint is incorrect), it outputs "Server is not running".
curl -f http://localhost:8000/ || echo "Server is not running"

# Display the docs in the notebook (rather than in the browser) - for fun

In [ ]:
# from IPython.display import IFrame

# # URL for the Swagger UI
# swagger_ui_url = "http://localhost:8000/docs"

# # Display Swagger UI in the notebook
# IFrame(src=swagger_ui_url, width='90%', height='600px')

# Query the service

**Using python**

In [5]:
import requests

# root query
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
response = requests.get("http://localhost:8000/", "", headers=headers)
print('Status Code:', response.status_code)
print('Response Text:', response.text)

2024-08-08 08:34:56,902 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-08-08 08:34:58,946 - urllib3.connectionpool - DEBUG - http://localhost:8000 "GET / HTTP/11" 200 78


Status Code: 200
Response Text: {"message":"Welcome 👋 to the Outlier Detection and Distribution Shift API"}


In [13]:
import requests
# outlier query
url = 'http://localhost:8000/anomaly/detect_outliers'
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
# convert input to json (fastapi will convert it automatically to proper format using pydantic. no need to serialize)
response = requests.post(url, json=payload.dict(), headers=headers)

# Check the status code and print appropriate messages
print('Status Code:', response.status_code)
if response.status_code == 200:
    try:
        print(response.json())
    except requests.exceptions.JSONDecodeError:
        print("Response content is not valid JSON")
else:
    print(f"Request failed with status code {response.status_code}")
    print(response.text)

2024-08-08 09:31:43,623 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:8000
2024-08-08 09:31:46,180 - urllib3.connectionpool - DEBUG - http://localhost:8000 "POST /anomaly/detect_outliers HTTP/11" 200 47


Status Code: 200
{'results': [{'is_outlier': 0}, {'is_outlier': 0}]}
